# Make necessary helper functions for comparing algorithms
* 02/21/23

In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import json

# local modules 
import sys
sys.path.insert(0, "../scripts")
from helpers import *

In [ ]:
# winow open / close detection metric + plotting 
# cross validation algorithm / way to try different parameter values quickly and see how the results change... => CV only really needed for ML 

In [ ]:
# for a given windows time series, have data about when window opened, and when window closed 
# because sometimes change freq of data, when win open/close should be decided after data imported 
# then can see (1) if window change detected w/in a given threshold 
# how far away detections are from reality 
# plot should highlight true window anomaly time + lines at the incidence
# for the algo, should show where made the guess...

In [6]:
r00, r01 = import_desired_data("A", "15T")

In [7]:
r00["Window Open"].unique()

array([1.])

In [8]:
r01["Window Open"].unique() 
# how to determine where the value is switching 

array([0., 1.])

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=r01["DateTime"],
    y=r01["Window Open"], 
    mode='lines+markers',
))

fig.add_trace(go.Scatter(
    x=r01["DateTime"],
    y=check*1 , 
    mode='markers',
))


In [12]:
# see where value is changing 
check = r01["Window Open"].shift() != r01["Window Open"]

In [17]:
check # True => 1 

0       True
1      False
2       True
3      False
4      False
       ...  
380    False
381    False
382    False
383    False
384    False
Name: Window Open, Length: 385, dtype: bool

In [16]:
check*1

0      1
1      0
2      1
3      0
4      0
      ..
380    0
381    0
382    0
383    0
384    0
Name: Window Open, Length: 385, dtype: int64

In [ ]:
# can see if the index that is passed in is 

In [26]:
c00, c01 = import_desired_data("C", "15T")
checkc = c00["Window Open"].shift() != c00["Window Open"]

In [27]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=c00["DateTime"],
    y=c00["Window Open"], 
    mode='lines+markers',
))

fig.add_trace(go.Scatter(
    x=c00["DateTime"],
    y=checkc*1 , 
    mode='markers',
))

In [28]:
# algos will return the index where they think the transition occured. metric should check if this index is true. if its not true, see the shortest distance from the nearest index 
len(checkc)

289

In [29]:
ix = 35

In [30]:
checkc[ix] # false 

False

In [ ]:
# find nearest true value => split before, after ix, first index in split array that is true 
# or indices of all ir , which has least abs dif.

In [34]:
# indices where we flip, skip the first b/c will always be "dif" with the shift method 
flips = np.where(check*1==1)[0][1:-1]
flips

array([  2,  21,  61, 120, 127, 137, 158, 194, 208, 227, 255])

In [35]:
# find nearest index to ix 
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [36]:
find_nearest(flips, ix)

21

In [37]:
# report distance (# time steps /indiced) from nearest 
ix - find_nearest(flips, ix)

14

In [ ]:
def calc_win_change_dist(df, ix ):
    # note where the value in Window Open series changes 
    shift  = df["Window Open"].shift() != df["Window Open"]
    
    # some of the experiments did not have any shifts 
    flips = np.where(check*1==1)[0] #[1:-1]
    if len(flips) < 2:
        return "No Flip Happened!"
    
    # drop the first entry which will always be true 
    flips = flips[1:-1]
